<a href="https://colab.research.google.com/github/IKosovych/ucu-recommender-system-2023/blob/master/Content_Based_Filtering__model_sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import pandas as pd
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
# Load the datasets
movies = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/ml-latest-small/movies.csv')
ratings = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/ml-latest-small/ratings.csv')
tags = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/ml-latest-small/tags.csv')
links = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/ml-latest-small/links.csv')


After inspection of four datases I didn't found severe outliers that can cause problems in our tasks.

###Content-Based Filtering model implementation

In [4]:
# Use TF-IDF to convert the genres into vectors
tfidf = TfidfVectorizer(stop_words='english')
movies['genres'] = movies['genres'].fillna('')
tfidf_matrix = tfidf.fit_transform(movies['genres'])

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [5]:
# Create a reverse mapping of movie titles and DataFrame indices
indices = pd.Series(movies.index, index=movies['title']).drop_duplicates()

def get_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwise similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return movies['title'].iloc[movie_indices]


In [6]:
print(get_recommendations('Toy Story (1995)'))


1706                                          Antz (1998)
2355                                   Toy Story 2 (1999)
2809       Adventures of Rocky and Bullwinkle, The (2000)
3000                     Emperor's New Groove, The (2000)
3568                                Monsters, Inc. (2001)
6194                                     Wild, The (2006)
6486                               Shrek the Third (2007)
6948                       Tale of Despereaux, The (2008)
7760    Asterix and the Vikings (Astérix et les Viking...
8219                                         Turbo (2013)
Name: title, dtype: object
